In [ ]:
# imports
import pyogrio
import geopandas as gpd
import pandas as pd
import dask_geopandas as dg
import dask.dataframe as dd
import cupy as cp
import networkx as nx
from dask.distributed import Client, LocalCluster
import os

In [2]:
# Define path to geodatabase
data_path = 'high_res_data/NHDPlus_H_National_Release_1_GDB/NHDPlus_H_National_Release_1_GDB.gdb'

In [ ]:
layers = pyogrio.list_layers(data_path)
layers

In [ ]:
cluster = LocalCluster(n_workers=4)
client = Client(cluster)
print(client.dashboard_link)

In [5]:
cat = dg.read_file(data_path, npartitions=32, layer='NHDPlusCatchment')
flow = dg.read_file(data_path, npartitions=32, layer='NHDPlusFlow')

In [6]:
cat = cat.get_partition(1)
flow = flow.get_partition(1)

In [ ]:
flow

In [ ]:
cat['geometry'].to_crs("epsg:5070")
cat.geometry = cat.geometry.centroid
area = cat.geometry.area.compute()
area

In [ ]:
print(flow.columns)

In [ ]:
# Create a directed graph for flow data

flow = flow.loc[:, ['frompermid', 'topermid']]
G = nx.DiGraph()
print(flow)
#G.add_edges_from((flow_from, flow_to))


In [ ]:
# Testing creating one massive dataframe
# could change layers to a dict where 
# key = (str) layer name
# val = (list[str]) important cols to load so we don't load excess data

df_list = []
for layer in layers:
    df = dg.read_file(data_path, npartitions=64, layer=layer)
    df_list.append(df)


In [ ]:
# Convert from geopandas to dask geopandas
dask_df = dd.concat(df_list)
#print(dask_df.shape)